In [6]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from MayaDataset import CNNCLF
import MayaDataset
from DefenderGAN import Warmup
from Models import RNNGenerator2, RNNInference
import argparse
import time
import os

dataset = MayaDataset.MayaDataset('raw_video',minpower=25, maxpower=225, window=1400, labels='video')
    
dsets = random_split(dataset, [1000,1000,1000, 1000])
trainset = dsets[0]
trainloader = DataLoader(trainset, batch_size=32, num_workers=4)

valset = dsets[1]
valloader = DataLoader(valset, batch_size=32, num_workers=4)

testset = dsets[2]
testloader = DataLoader(testset, batch_size=32, num_workers=4)

dim=64
clf = CNNCLF(dataset.window).cuda()
gen = RNNGenerator2(dim).cuda()
bestname = './best_{}_{}_{}.pth'.format('video','rnn2',dim)
if os.path.isfile(bestname):
    print('Previous best found: loading the model...')
    gen.load_state_dict(torch.load(bestname))
gen.train()


Previous best found: loading the model...


RNNGenerator2(
  (encoder): Sequential(
    (0): NoiseInjector()
    (1): Conv1d(1, 64, kernel_size=(32,), stride=(1,))
    (2): ReLU()
    (3): NoiseInjector()
  )
  (resblock): GRU(64, 64, batch_first=True)
  (decoder): Sequential(
    (0): NoiseInjector()
    (1): Linear(in_features=64, out_features=1, bias=True)
    (2): Hardtanh(min_val=-1.0, max_val=1.0)
  )
)

In [7]:
optim_c = torch.optim.Adam(clf.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
bestacc = 0.0
epochs = 30
for e in range(epochs):
    clf.train()
    for x,y in trainloader:
        optim_c.zero_grad()
        xdata, ydata = x.cuda(), y.cuda()
        perturb = gen(xdata)
        p_input = torch.relu(xdata + perturb.detach())
        output = clf(p_input)
        loss = criterion(output, ydata)
        loss.backward()
        optim_c.step()
    totcorrect = 0
    totcount = 0
    clf.eval()
    orgpower = 0.0
    newpower = 0.0
    for x,y in valloader:
        xdata, ydata = x.cuda(), y.cuda()
        orgpower += xdata.mean().item()
        perturb = gen(xdata)
        p_input = torch.relu(xdata + perturb.detach())
        newpower += p_input.mean().item()
        output = clf(p_input)
        pred = output.argmax(axis=-1)
        totcorrect += (pred==ydata).sum().item()
        totcount += y.size(0)
    macc = float(totcorrect)/totcount
    orgpower = orgpower/len(valloader)
    orgpower = orgpower*200 + 25
    newpower = newpower/len(valloader)
    newpower = newpower*200 + 25
    if(macc > bestacc) and e > epochs//2:
        bestacc = macc
        bestclf = clf.state_dict()
    print("Cooldown {}\t acc {:.4f}\torgpower {:.2f}\t newpower: {:.2f}".format(e+1, macc, orgpower, newpower))

clf.load_state_dict(bestclf)
totcorrect = 0
totcount = 0
clf.eval()
orgpower = 0.0
newpower = 0.0
for x,y in testloader:
    xdata, ydata = x.cuda(), y.cuda()
    orgpower += xdata.mean().item()
    perturb = gen(xdata)
    p_input = torch.relu(xdata + perturb.detach())
    newpower += p_input.mean().item()
    output = clf(p_input)
    pred = output.argmax(axis=-1)
    totcorrect += (pred==ydata).sum().item()
    totcount += y.size(0)
macc = float(totcorrect)/totcount
orgpower = orgpower/len(valloader)
orgpower = orgpower*200 + 25
newpower = newpower/len(valloader)
newpower = newpower*200 + 25
print("Test acc {:.4f}\torgpower {:.2f}\t newpower: {:.2f}".format(macc, orgpower, newpower))

Cooldown 1	 acc 0.1810	orgpower 80.13	 newpower: 79.91
Cooldown 2	 acc 0.1980	orgpower 80.13	 newpower: 79.91
Cooldown 3	 acc 0.2130	orgpower 80.13	 newpower: 79.90
Cooldown 4	 acc 0.2220	orgpower 80.13	 newpower: 79.91
Cooldown 5	 acc 0.2400	orgpower 80.13	 newpower: 79.91
Cooldown 6	 acc 0.2390	orgpower 80.13	 newpower: 79.91
Cooldown 7	 acc 0.2680	orgpower 80.13	 newpower: 79.91
Cooldown 8	 acc 0.2460	orgpower 80.13	 newpower: 79.91
Cooldown 9	 acc 0.2330	orgpower 80.13	 newpower: 79.90
Cooldown 10	 acc 0.2500	orgpower 80.13	 newpower: 79.91
Cooldown 11	 acc 0.2350	orgpower 80.13	 newpower: 79.91
Cooldown 12	 acc 0.2560	orgpower 80.13	 newpower: 79.90
Cooldown 13	 acc 0.2500	orgpower 80.13	 newpower: 79.91
Cooldown 14	 acc 0.2430	orgpower 80.13	 newpower: 79.91
Cooldown 15	 acc 0.2530	orgpower 80.13	 newpower: 79.90
Cooldown 16	 acc 0.2630	orgpower 80.13	 newpower: 79.91
Cooldown 17	 acc 0.2230	orgpower 80.13	 newpower: 79.91
Cooldown 18	 acc 0.2530	orgpower 80.13	 newpower: 79.91
C

In [ ]:
clf.load_state_dict(bestclf)
totcorrect = 0
totcount = 0
clf.eval()
orgpower = 0.0
newpower = 0.0
for x,y in testloader:
    xdata, ydata = x.cuda(), y.cuda()
    orgpower += xdata.mean().item()
    perturb = gen(xdata)
    p_input = torch.relu(xdata + perturb.detach())
    newpower += p_input.mean().item()
    output = clf(p_input)
    pred = output.argmax(axis=-1)
    totcorrect += (pred==ydata).sum().item()
    totcount += y.size(0)
macc = float(totcorrect)/totcount
orgpower = orgpower/len(valloader)
orgpower = orgpower*200 + 25
newpower = newpower/len(valloader)
newpower = newpower*200 + 25
print("Test acc {:.4f}\torgpower {:.2f}\t newpower: {:.2f}".format(macc, orgpower, newpower))

In [ ]:
train_x = []
train_y = []
valid_x = []
valid_y = []
for x,y in trainloader:
    xdata, ydata = x.cuda(), y.cuda()
    perturb = gen(xdata)
    p_input = torch.relu(xdata + perturb.detach())
    for p in p_input.cpu():
        train_x.append(p.numpy())
    for label in y:
        train_y.append(label)

for x,y in testloader:
    xdata, ydata = x.cuda(), y.cuda()
    perturb = gen(xdata)
    p_input = torch.relu(xdata + perturb.detach())
    for p in p_input.cpu():
        valid_x.append(p.numpy())
    for label in y:
        valid_y.append(label)
    

In [ ]:

from sklearn import svm
clf = svm.LinearSVC()
clf.fit(train_x, train_y)
pred_y = clf.predict(valid_x)

In [ ]:
correct = [yp == y for yp,y in zip(pred_y, valid_y)]
sum(correct)/len(correct)

In [ ]:
print(clf.coef_.shape)
print(clf.intercept_.shape)

In [8]:
model = RNNInference(dim)
model.copy_params(gen.cpu())
script_module = torch.jit.trace(model, (torch.rand(1,32), torch.rand(1,dim)))

/home/hwnam/anaconda3/envs/py38/lib/python3.8/site-packages/torch/jit/_trace.py:983: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1 / 1 (100.0%)
Greatest absolute difference: 0.12589895725250244 at index (0, 0) (up to 1e-05 allowed)
Greatest relative difference: 0.2129104169799826 at index (0, 0) (up to 1e-05 allowed)
  _check_trace(
/home/hwnam/anaconda3/envs/py38/lib/python3.8/site-packages/torch/jit/_trace.py:983: TracerWarning: Output nr 2. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 64 / 64 (100.0%)
Greatest absolute difference: 0.04112176597118378 at index (0, 32) (up to 1e-05 allowed)
Greatest relative difference: 22.26038532466272 at index (0, 25) (up to 1e-05 allowed)
  _check_trace(


In [ ]:
script_module.save('./cpuscript_{}_{}_{}.pt'.format('video','rnn2',dim))

In [ ]:
aa = nn.Linear(32,64)
model.encoder[1].weight.shape
